In [1]:
from bystro.api import auth
from bystro.proteomics.annotation_interface import get_annotation_result_from_query, execute_query, async_get_annotation_result_from_query

In [ ]:
user = auth.login('email', 'password', 'https://bystro-dev.emory.edu')
index = '663ac4bfa0e17a1660ba8130_657a50d4b2d0278938ba791d'

In [13]:
# Default behavior is to show 1 sample per row, e.g. `melt_samples=True`
query_result_df_array_of_structs_select_fields = get_annotation_result_from_query(
    query_string="*",
    index_name=index,
    bystro_api_auth=user,
    fields=['gnomad.exomes.AF', 'gnomad.genomes.AF']
)
query_result_df_array_of_structs_select_fields.head(n=10)

,chrom,pos,vcfPos,inputRef,alt,type,id,locus,sample,dosage,gnomad.exomes.AF,gnomad.genomes.AF
0,chr17,2923767,2923767,A,G,SNP,.,chr17:2923767:A:G,1805,2,0.918969,0.922388
1,chr17,2923767,2923767,A,G,SNP,.,chr17:2923767:A:G,1847,2,0.918969,0.922388
2,chr17,2923767,2923767,A,G,SNP,.,chr17:2923767:A:G,4805,2,0.918969,0.922388
3,chr17,2929286,2929286,A,C,SNP,.,chr17:2929286:A:C,1805,2,0.686967,0.695134
4,chr17,2929286,2929286,A,C,SNP,.,chr17:2929286:A:C,1847,2,0.686967,0.695134
5,chr17,2929286,2929286,A,C,SNP,.,chr17:2929286:A:C,4805,2,0.686967,0.695134
6,chr17,2929392,2929392,G,A,SNP,.,chr17:2929392:G:A,1805,1,0.441199,0.393499
7,chr17,2929392,2929392,G,A,SNP,.,chr17:2929392:G:A,1847,1,0.441199,0.393499
8,chr17,2929392,2929392,G,A,SNP,.,chr17:2929392:G:A,4805,2,0.441199,0.393499
9,chr2,196825402,196825402,T,C,SNP,.,chr2:196825402:T:C,1847,1,0.131093,0.157932


In [14]:
# You can choose to also "melt" the dataframe by a field,
# so that when the field has an array value, it is flattened
# When the primary key of the track isn't found, by default we force that 
# field to have 1 value per row, and warn you of this behavior.
# To change it, set `force_flatten_exploded_field=False`
query_result_df = get_annotation_result_from_query(
    query_string="(type:del && (alt:\-6 || alt:\-1)) || (cadd:>40)",
    index_name=index,
    bystro_api_auth=user,
    melt_samples=True,
    explode_field='refSeq.name2',
    fields=['refSeq.name2', 'refSeq.spID', 'refSeq.name'],
    force_flatten_exploded_field=False
)
query_result_df[['refSeq.name2', 'refSeq.spID']]

,refSeq.name2,refSeq.spID
0,SENP3,Q9H4L4
1,SENP3-EIF4A1,None
2,SENP3,Q9H4L4
3,SENP3-EIF4A1,None
4,SENP3,Q9H4L4
...,...,...
1893,DTYMK,None
1894,DTYMK,None
1895,DTYMK,None
1896,ING5,"[B7Z6R2, Q8WYH8]"


In [15]:
# You can disable sample melting by setting `melt_samples=False`
query_result_df = await async_get_annotation_result_from_query(
    query_string="(type:del && (alt:\-6 || alt:\-1)) || (cadd:>40)",
    index_name=index,
    bystro_api_auth=user,
    fields=['refSeq.name2', 'refSeq.name'],
    melt_samples=False,
    explode_field='refSeq.name2'
)
query_result_df.head(n=10)

,chrom,pos,vcfPos,inputRef,alt,type,id,locus,refSeq.name2,refSeq.name
0,chr17,7470286,7470285,A,-1,DEL,.,chr17:7470286:A:-1,SENP3,NM_015670
1,chr17,7470286,7470285,A,-1,DEL,.,chr17:7470286:A:-1,SENP3-EIF4A1,NR_037926
2,chr2,220427429,220427428,A,-1,DEL,.,chr2:220427429:A:-1,OBSL1,NM_015311
3,chr2,220427429,220427428,A,-1,DEL,.,chr2:220427429:A:-1,OBSL1,NM_001173431
4,chr2,220427429,220427428,A,-1,DEL,.,chr2:220427429:A:-1,OBSL1,NM_001173408
5,chr2,238835573,238835572,T,-1,DEL,.,chr2:238835573:T:-1,None,None
6,chr15,20490550,20490549,T,-1,DEL,.,chr15:20490550:T:-1,CHEK2P2,NR_038836
7,chr15,23685542,23685541,T,-1,DEL,.,chr15:23685542:T:-1,GOLGA6L2,NM_001304388
8,chr15,25453230,25453229,G,-1,DEL,.,chr15:25453230:G:-1,SNHG14,NR_146177
9,chr15,25453230,25453229,G,-1,DEL,.,chr15:25453230:G:-1,SNORD115-21,NR_003313


In [4]:
# You can also represent tracks that have multiple fields by dictionaries, instead of flattening
# the dictionaries into separate columns, by setting `structs_of_arrays=False`
# This is currently incompatible with `explode_field` and `fields`
query_result_df_array_of_structs_no_melt = get_annotation_result_from_query(
    query_string="*",
    index_name=index,
    bystro_api_auth=user,
    structs_of_arrays=False
)
query_result_df_array_of_structs_no_melt.head(n=10)

,chrom,pos,vcfPos,inputRef,alt,type,id,locus,sample,dosage,...,gnomad.genomes,heterozygosity,homozygosity,missingness,nearest.refSeq,nearestTss.refSeq,ref,refSeq,sampleMaf,trTv
0,chr17,2923767,2923767,A,G,SNP,.,chr17:2923767:A:G,1805,2,...,"{'AN_nfe_nwe': 8598, 'AF_fin': 0.9588609933853...",0.000,1.000,0.0,"[{'name2': 'RAP1GAP2', 'name': 'NM_001330058',...","[{'name2': 'OR1D5', 'name': 'NM_014566', 'dist...",A,"[{'kgID': None, 'spDisplayID': None, 'rfamAcc'...",1.000,1
1,chr17,2923767,2923767,A,G,SNP,.,chr17:2923767:A:G,1847,2,...,"{'AN_nfe_nwe': 8598, 'AF_fin': 0.9588609933853...",0.000,1.000,0.0,"[{'name2': 'RAP1GAP2', 'name': 'NM_001330058',...","[{'name2': 'OR1D5', 'name': 'NM_014566', 'dist...",A,"[{'kgID': None, 'spDisplayID': None, 'rfamAcc'...",1.000,1
2,chr17,2923767,2923767,A,G,SNP,.,chr17:2923767:A:G,4805,2,...,"{'AN_nfe_nwe': 8598, 'AF_fin': 0.9588609933853...",0.000,1.000,0.0,"[{'name2': 'RAP1GAP2', 'name': 'NM_001330058',...","[{'name2': 'OR1D5', 'name': 'NM_014566', 'dist...",A,"[{'kgID': None, 'spDisplayID': None, 'rfamAcc'...",1.000,1
3,chr17,2929286,2929286,A,C,SNP,.,chr17:2929286:A:C,1805,2,...,"{'AF_nfe_seu': 0.537735998630524, 'AF_afr': 0....",0.000,1.000,0.0,"[{'name': 'NM_001330058', 'dist': 0, 'name2': ...","[{'name2': 'OR1D5', 'name': 'NM_014566', 'dist...",A,"[{'siteType': 'intronic', 'altAminoAcid': None...",1.000,2
4,chr17,2929286,2929286,A,C,SNP,.,chr17:2929286:A:C,1847,2,...,"{'AF_nfe_seu': 0.537735998630524, 'AF_afr': 0....",0.000,1.000,0.0,"[{'name': 'NM_001330058', 'dist': 0, 'name2': ...","[{'name2': 'OR1D5', 'name': 'NM_014566', 'dist...",A,"[{'siteType': 'intronic', 'altAminoAcid': None...",1.000,2
5,chr17,2929286,2929286,A,C,SNP,.,chr17:2929286:A:C,4805,2,...,"{'AF_nfe_seu': 0.537735998630524, 'AF_afr': 0....",0.000,1.000,0.0,"[{'name': 'NM_001330058', 'dist': 0, 'name2': ...","[{'name2': 'OR1D5', 'name': 'NM_014566', 'dist...",A,"[{'siteType': 'intronic', 'altAminoAcid': None...",1.000,2
6,chr17,2929392,2929392,G,A,SNP,.,chr17:2929392:G:A,1805,1,...,"{'AF_afr': 0.270516008138657, 'AF_fin': 0.4530...",0.667,0.333,0.0,"[{'name2': 'RAP1GAP2', 'name': 'NM_001330058',...","[{'name': 'NM_014566', 'dist': 36571, 'name2':...",G,"[{'exonicAlleleFunction': 'synonymous', 'altCo...",0.667,1
7,chr17,2929392,2929392,G,A,SNP,.,chr17:2929392:G:A,1847,1,...,"{'AF_afr': 0.270516008138657, 'AF_fin': 0.4530...",0.667,0.333,0.0,"[{'name2': 'RAP1GAP2', 'name': 'NM_001330058',...","[{'name': 'NM_014566', 'dist': 36571, 'name2':...",G,"[{'exonicAlleleFunction': 'synonymous', 'altCo...",0.667,1
8,chr17,2929392,2929392,G,A,SNP,.,chr17:2929392:G:A,4805,2,...,"{'AF_afr': 0.270516008138657, 'AF_fin': 0.4530...",0.667,0.333,0.0,"[{'name2': 'RAP1GAP2', 'name': 'NM_001330058',...","[{'name': 'NM_014566', 'dist': 36571, 'name2':...",G,"[{'exonicAlleleFunction': 'synonymous', 'altCo...",0.667,1
9,chr2,196825402,196825402,T,C,SNP,.,chr2:196825402:T:C,1847,1,...,"{'AN_nfe': 15422, 'AF_nfe': 0.123718999326229,...",0.333,0.000,0.0,"[{'name2': 'DNAH7', 'name': 'NM_018897', 'dist...","[{'name2': 'STK17B', 'name': 'NM_004226', 'dis...",T,"[{'refAminoAcid': 'K', 'kgID': 'uc002utj.4', '...",0.167,1
